# Import Packages

In [7]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import os
from numpy import sort
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from collections import Counter
import math 
from gensim.models import Word2Vec
import gc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# EDA and Data Pre-processing
## Read Data

In [41]:
trl = pd.read_csv('train_label.csv')
trb = pd.read_csv('train_base.csv')
tro = pd.read_csv('train_op.csv')
trt = pd.read_csv('train_trans.csv')

teb = pd.read_csv('testb_base.csv')
teo = pd.read_csv('testb_op.csv')
tet = pd.read_csv('testb_trans.csv')

trbl = pd.merge(trb, trl, on = 'user', how = 'left')
data = pd.concat([trbl, teb], axis=0, ignore_index=True)

op = pd.concat([tro, teo], axis=0, ignore_index=True)
trans = pd.concat([trt, tet], axis=0, ignore_index=True)

del trl, trb, tro, trt, teb, teo, tet, trbl

In [19]:
op.head()
trans.head()
data.head()

,user,op_type,op_mode,op_device,ip,net_type,channel,ip_3,tm_diff
0,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:22.000000000
1,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:21.000000000
2,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:23.000000000
3,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:26.000000000
4,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:41.000000000


,user,platform,tunnel_in,tunnel_out,amount,type1,ip,type2,ip_3,tm_diff
0,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,185784,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:02:45.000000000
1,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,391769,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:03:58.000000000
2,Train_08351,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,27740,674e8d5860bc033d,f10a09fe9e522a47,11a213398ee0c623,ee386d6f9fe45d0d,18 days 11:06:49.000000000
3,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,36689,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 09:52:51.000000000
4,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,30746,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 07:50:05.000000000


,user,sex,age,provider,level,verified,using_time,regist_type,card_a_cnt,card_b_cnt,...,service3_level,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,product7_cnt,product7_fail_cnt,label
0,Train_06800,category 1,24877,category 0,category 2,category 0,24728,category 1,24712,24712,...,NaN,level 1,level 1,level 1,level 0,level 0,level 1,24706,24706,0.0
1,Train_23487,category 1,24895,category 0,category 2,category 0,24719,category 1,24719,24719,...,NaN,level 1,level 1,level 1,level 0,level 0,level 1,24712,24706,1.0
2,Train_36880,category 0,24853,category 0,category 2,category 0,24723,category 1,24719,24719,...,NaN,level 1,level 1,level 1,level 0,level 0,level 1,24712,24706,1.0
3,Train_35392,category 0,24938,category 1,category 1,category 0,24707,category 3,24712,24712,...,NaN,level 1,level 1,level 1,level 0,level 0,level 1,24712,24706,0.0
4,Train_35057,category 0,24956,category 0,category 2,category 0,24728,category 2,24719,24712,...,NaN,level 2,level 8,level 1,level 0,level 0,level 1,24712,24706,1.0


# Feature Engineering
Time + Amount + Recency + Equipment + Geography + Stability

In [20]:
def fill_na_zero(nadata, col1, col2):
    idx = nadata[nadata[col2]==0].index
    nadata.loc[idx, col1] = 0
    return nadata


def fill_discrete_conti_na(nadata):
    i = 0
    for col in nadata.columns:
        i += 1
        if i%10 == 0:
            #print(i)
        if col in ['user', 'label']:
            continue
        if nadata[col].dtype in ['int64','float64', 'int32', 'float32']:
            num = len(set(nadata[col]))
            if num < 10: # 离散型
                #print(col)
                nadata[col] = nadata[col].fillna(nadata[col].mode())
                nadata[col] = nadata[col].astype(float)
            else:
                nadata[col] = nadata[col].fillna(nadata[col].median())
                nadata[col] = nadata[col].astype(float)
    return nadata  


def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

## Transaction related features

In [38]:
# time related
op['days'] = op['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
op['week'] = op['days'].apply(lambda x: x % 7)
trans['days'] = trans['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
trans['week'] = trans['days'].apply(lambda x: x % 7)

trans['days_diff'] = trans.groupby('user')['days'].diff()
op['days_diff'] = op.groupby('user')['days'].diff()

op['hour'] = op['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
trans['hour'] = trans['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))

# money amount related
temp = trans.groupby(['user'])['amount'].agg(
        user_amount_mean = 'mean',
        user_amount_std = 'std',
        user_amount_max = 'max',
        user_amount_min = 'min',
        user_amount_sum = 'sum',
        user_amount_med = 'median',
        user_amount_cnt = 'count',
        ).reset_index()

data = pd.merge(data, temp, on = 'user', how = 'left')

data['user_trans_days_nuniq'] = trans.groupby(['user'])['days'].nunique()


data['user_amount_per_days'] = data['user_amount_sum'] / data['user_trans_days_nuniq']
data = fill_na_zero(data, 'user_amount_per_days', 'user_trans_days_nuniq')
data['user_amount_per_cnt'] = data['user_amount_sum'] / data['user_amount_cnt']
data = fill_na_zero(data, 'user_amount_per_cnt', 'user_amount_cnt')



for day in [7, 15, 23, 27]:
    temp = group_data = trans[trans['days']>day].groupby('user')['amount'].agg(
        col1='mean',
        col2='std',
        col3='max',
        col4='min',
        col5='sum',
        col6='median',
        col7='count',
        ).reset_index().rename(columns = {'col1': 'user_amount_mean_{}d'.format(day), 'col2': 'user_amount_std_{}d'.format(day), 
        'col3':'user_amount_max_{}d'.format(day), 'col4': 'user_amount_min_{}d'.format(day), 'col5': 'user_amount_sum_{}d'.format(day), 
        'col6':'user_amount_med_{}d'.format(day), 'col7': 'user_amount_cnt_{}d'.format(day)})
data = pd.merge(data, temp, on='user', how='left')


win = [0, 15, 31]
for i in range(len(win)-1):
    temp = trans[(trans['days_diff']>=win[i])&(trans['days_diff']<win[i+1])].groupby('user')['amount'].agg(
        col1='mean',
        col2='std',
        col3='max',
        col4='min',
        col5='sum',
        col6='median',
        col7='count',
        ).reset_index().rename(columns = {'col1': 'user_amount_mean_{}_{}d'.format(win[i], win[i+1]), 'col2': 'user_amount_std_{}_{}d'.format(win[i], win[i+1]), 
        'col3':'user_amount_max_{}_{}d'.format(win[i], win[i+1]), 'col4': 'user_amount_min_{}_{}d'.format(win[i], win[i+1]), 'col5': 'user_amount_sum_{}_{}d'.format(win[i], win[i+1]), 
        'col6':'user_amount_med_{}_{}d'.format(win[i], win[i+1]), 'col7': 'user_amount_cnt_{}_{}d'.format(win[i], win[i+1])})
data = pd.merge(data, temp, on = "user", how = 'left')


# categorical variables
for col in ['platform', 'tunnel_in', 'tunnel_out', 'type1', 'type2', 'ip', 'ip_3', 'days']:
    temp = trans.groupby(['user'])[col].agg(col = 'nunique').reset_index().rename(columns = {'col': 'user_{}_nuniq_{}'.format(col, 'trans')})
    data = data.merge(temp, on=['user'], how='left')

for col in ['platform', 'type1', 'type2']:
    temp = trans.pivot_table(index='user', columns=col, values='amount', dropna=False, aggfunc=['count', 'sum']).fillna(0)
    temp.columns = ['user_{}_{}_amount_{}'.format(col, t[1], t[0]) for t in temp.columns]
    temp = temp.reset_index()
    data = pd.merge(data, temp, on='user', how='left')

temp = data.groupby(['user'])['city'].agg(city_cnt = 'count').reset_index()
data = pd.merge(data, temp, on = 'user', how = 'left')
temp = data.groupby(['user'])['province'].agg(province_cnt = 'count').reset_index()
data = pd.merge(data, temp, on = 'user', how = 'left')

for col in [t for t in data.select_dtypes('object').columns if t not in ['user']]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# transaction fail ratio
data['product7_fail_ratio'] = data['product7_fail_cnt'] / data['product7_cnt']
data = fill_na_zero(data, 'product7_fail_ratio', 'product7_cnt')


# transaction amount difference
data['user_amount_gap'] = data['user_amount_max']-data['user_amount_min']

data['user_trans_days_diff_nuniq'] = trans.groupby(['user'])['days_diff'].nunique()

# average transaction times
data['user_count_per_days'] = data['user_amount_cnt']/data['user_trans_days_diff_nuniq']
data = fill_na_zero(data, 'user_count_per_days', 'user_trans_days_diff_nuniq')

# average times of transaction platform, type1 and type2 used
for value in ['platform', 'type1', 'type2']:
    fea = list(set(trans[value]))
    for f in fea:
        if str(f)!= 'nan':
            data['user_{}_{}_amount_perday_count'.format(value, f)] = data['user_{}_{}_amount_count'.format(value, f)]/data['user_trans_days_diff_nuniq']
            data = fill_na_zero(data, 'user_{}_{}_amount_perday_count'.format(value, f), 'user_trans_days_diff_nuniq')

# average transaction money amount of transaction platform, type1 and type2
for value in ['platform', 'type1', 'type2']:
    fea = list(set(trans[value]))
    for f in fea:
        if str(f)!= 'nan':
            data['user_{}_{}_amount_perday'.format(value, f)] = data['user_{}_{}_amount_sum'.format(value, f)]/data['user_trans_days_diff_nuniq']
            data = fill_na_zero(data, 'user_{}_{}_amount_perday'.format(value, f), 'user_trans_days_diff_nuniq')
            data['user_{}_{}_amount_percount'.format(value, f)] = data['user_{}_{}_amount_sum'.format(value, f)]/data['user_amount_cnt']
            data = fill_na_zero(data, 'user_{}_{}_amount_percount'.format(value, f), 'user_amount_cnt')

# transaction gap days
for value in ['platform', 'type1', 'type2']:
    fea = list(set(trans[value]))
    for f in fea:
        if str(f)!= 'nan':
            group_data = trans[trans[value]==f].groupby(['user'])['days_diff'].agg(col1 = 'min', col2 = 'max').reset_index().rename(columns = {'col1': 'user_{}_{}_min_day'.format(value, f), 'col2': 'user_{}_{}_max_day'.format(value, f)})
            group_data['user_{}_{}_Is_recent_trd_mean'.format(value, f)]=(group_data['user_{}_{}_max_day'.format(value, f)]>group_data['user_{}_{}_max_day'.format(value, f)].mean())*1
            group_data['user_{}_{}_gap_day'.format(value, f)] = group_data['user_{}_{}_max_day'.format(value, f)]-group_data['user_{}_{}_min_day'.format(value, f)]
            data = data.merge(group_data, on=['user'], how='left')
            #data = data.merge(group_data[['user','user_{}_{}_gap_day'.format(value, f)]], on=['user'], how='left')
            #data['user_{}_{}_gap_day'.format(value, f)] = data['user_{}_{}_max_day'.format(value, f)]-data['user_{}_{}_min_day'.format(value, f)]



# transaction gap money amount
for value in ['platform', 'type1', 'type2']:
    fea = list(set(trans[value]))
    for f in fea:
        if str(f)!= 'nan':
            group_data = trans[trans[value]==f].groupby(['user'])['amount'].agg(col1 = 'min', col2 = 'max').reset_index().rename(columns = {'col1': 'user_{}_{}_min_amount'.format(value, f), 'col2':'user_{}_{}_max_amount'.format(value, f)})
            data = data.merge(group_data, on=['user'], how='left')
            data['user_{}_{}_gap_amount'.format(value, f)] = data['user_{}_{}_max_amount'.format(value, f)]-data['user_{}_{}_min_amount'.format(value, f)]

recent_trd = trans.groupby(['user'])['days_diff'].agg(recent_trd = 'max').reset_index()

recent_trd['Is_recent_trd_mean']=(recent_trd['recent_trd']>recent_trd['recent_trd'].mean())*1

data = pd.merge(data, recent_trd, how='left', on='user')


unique_col = list(set(data.columns))
data = data[data.columns].T.drop_duplicates().T

trans['hour_group'] = pd.cut(trans['hour'], [-1, 6, 12, 18, 24])

# Time Period - User Habit Exploration
for value in ['hour_group']:
    fea = list(set(trans[value]))
    for f in fea:
        if str(f)!= 'nan':
            temp = str(f).replace(",", "_")
            temp = temp[1:-1]
            group_data = trans[trans[value]==f].groupby(['user'])['amount'].agg(col1 = 'min', col2 = 'max', col3 = 'mean', col4 = 'count').reset_index().rename(columns = {'col1': 'user_{}_{}_min_amount'.format(value, temp), 
                               'col2':'user_{}_{}_max_amount'.format(value, temp), 'col3':'user_{}_{}_mean_amount'.format(value, temp), 'col4': 'user_{}_{}_cnt_amount'.format(value, temp)})
            data = data.merge(group_data, on=['user'], how='left')
            data['user_{}_{}_gap_amount'.format(value, temp)] = data['user_{}_{}_max_amount'.format(value, temp)]-data['user_{}_{}_min_amount'.format(value, temp)]

# Statistical characteristics by week
for value in ['week']:
    fea = list(set(trans[value]))
    for f in fea:
        if str(f)!= 'nan':
            group_data = trans[trans[value]==f].groupby(['user'])['amount'].agg(col1 = 'min', col2 = 'max', col3 = 'mean', col4 = 'count', col5 = 'sum', col6 = 'median', col7 = 'std', col8 = percentile(25), col9 = percentile(75)).reset_index().rename(columns = {'col1': 'user_{}_{}_min_amount'.format(value, f),
                               'col2':'user_{}_{}_max_amount'.format(value, f), 'col3':'user_{}_{}_mean_amount'.format(value, f), 'col4': 'user_{}_{}_cnt_amount'.format(value, f),
                               'col5':'user_{}_{}_sum_amount'.format(value, f), 'col6':'user_{}_{}_median_amount'.format(value, f), 'col7': 'user_{}_{}_std_amount'.format(value, f),
                               'col8':'user_{}_{}_25_amount'.format(value, f), 'col9': 'user_{}_{}_75_amount'.format(value, f)})
            #group_data['user_{}_{}_25_amount'.format(value, f)] = trans[trans[value]==f].groupby(['user']).agg(lambda x: np.percentile(x['amount'], q = 25))
            data = data.merge(group_data, on=['user'], how='left')
            data['user_{}_{}_gap_amount'.format(value, f)] = data['user_{}_{}_max_amount'.format(value, f)]-data['user_{}_{}_min_amount'.format(value, f)]

# Number of trading days per week, average daily trade amount per week, number of trades
for value in ['week']:
    fea = list(set(trans[value]))
    for f in fea:
        if str(f)!= 'nan':
            group_data = trans[trans[value]==f].groupby(['user'])['days_diff'].agg(col1 = 'nunique').reset_index().rename(columns = {'col1': 'user_{}_{}_nuniq_days'.format(value, f)})
            data = data.merge(group_data, on=['user'], how='left')
            data['user_{}_{}_amount_per_days'.format(value, f)] = data['user_{}_{}_sum_amount'.format(value, f)]/data['user_{}_{}_nuniq_days'.format(value, f)]
            data = fill_na_zero(data, 'user_{}_{}_amount_per_days'.format(value, f), 'user_{}_{}_nuniq_days'.format(value, f))
            data['user_{}_{}_times_per_days'.format(value, f)] = data['user_{}_{}_cnt_amount'.format(value, f)]/data['user_{}_{}_nuniq_days'.format(value, f)]
            data = fill_na_zero(data, 'user_{}_{}_times_per_days'.format(value, f), 'user_{}_{}_nuniq_days'.format(value, f))

for op_col in ['min', 'max', 'mean', 'cnt', 'sum', 'median', 'std', '25', '75', 'gap']:
    col = []
    for w in list(set(trans['week'])):
        col.append('user_week_{}_{}_amount'.format(w, op_col))
    temp = data[col]
    data['user_allweek_{}_mean_amount'.format(op_col)] = np.nanmean(temp, axis = 1)
    data['user_allweek_{}_std_amount'.format(op_col)] = np.nanstd(temp, axis = 1)


for op_col in ['nuniq', 'amount_per', 'times_per']:
    col = []
    for w in list(set(trans['week'])):
        col.append('user_week_{}_{}_days'.format(w, op_col))
    temp = data[col]
    data['user_allweek_{}_mean'.format(op_col)] = np.nanmean(temp, axis = 1)
    data['user_allweek_{}_std'.format(op_col)] = np.nanstd(temp, axis = 1)

# 25,75th percentile of the money amount
group_data = trans.groupby(['user'])['amount'].agg(
        user_amount_25 = percentile(25),
        user_amount_75 = percentile(75)
        ).reset_index()
data = data.merge(group_data, on=['user'], how='left')


# 25, 75th percentile of the amount of the time window
for time_w in [7, 15, 23, 27]:
    group_data = trans[trans['days_diff']>time_w].groupby('user')['amount'].agg(
            col1 = percentile(25),
            col2 = percentile(75)
            ).reset_index().rename(columns = {'col1': 'user_amount_25_{}d'.format(time_w), 'col2': 'user_amount_75_{}d'.format(time_w)})
    data = data.merge(group_data, on=['user'], how='left')

for x in ['op_device', 'ip', 'net_type']:
    op['is_null'] = 0
    op.loc[op[x].isnull(), 'is_null'] = 1

    temp = op.groupby(['user'])['is_null'].agg(
            col1 = 'sum',
            col2 = 'mean').reset_index().rename(columns = {'col1': 'user_{}_{}_null_cnt'.format('op', x), 'col2':'user_{}_{}_null_ratio'.format('op', x)})
    data = pd.merge(data, temp, on = 'user', how = 'left')

temp = op.groupby('user')['net_type'].agg(op_times = 'count').reset_index()
data = pd.merge(data, temp, on=['user'], how='left')

# net
for x in list(set(op['net_type'])):
    if str(temp) == 'nan':
        continue
    temp = op[op['net_type']==x].groupby('user')['net_type'].agg(col1 = 'count').reset_index().rename(columns = {'col1': 'user_{}_cnt'.format(x)})
    data = pd.merge(data, temp, on=['user'], how='left')
    data['user_{}_ratio'.format(x)] = data['user_{}_cnt'.format(x)]/data['op_times']

# Aggregation degree
def radio_d(x):
    temp = Counter(x)
    return max(list(temp.values()))/sum(temp.values())

def radio_d_two(x):
    temp = Counter(x)
    if len(list(temp.values()))>=2:
        return sum(sorted(list(temp.values()), reverse = True)[:2])/sum(temp.values())
    else:
        return max(list(temp.values()))/sum(temp.values())
    
def radio_d_three(x):
    temp = Counter(x)
    if len(list(temp.values()))>=3:
        return sum(sorted(list(temp.values()), reverse = True)[:3])/sum(temp.values())
    elif len(list(temp.values()))>=2:
        return sum(sorted(list(temp.values()), reverse = True)[:2])/sum(temp.values())
    else:
        return max(list(temp.values()))/sum(temp.values())

# user aggregation
temp = op.groupby(['user'])['days_diff'].agg(
            user_centre_1d = radio_d,
            user_centre_2d = radio_d_two,
            user_centre_3d = radio_d_three
        ).reset_index()
data = pd.merge(data, temp, on=['user'], how='left')


'''
temp = Counter(trans['days_diff'])
tmp = max(list(temp.values()))/sum(temp.values())
sum(sorted(list(temp.values()), reverse = True)[:2])/sum(temp.values())


def radio_d(x):
    temp = Counter(x)
    #print(temp)
    return max(list(temp.values()))/sum(temp.values())

def radio_d_two(x):
    temp = Counter(x)
    if len(list(temp.values()))>=2:
        return sum(sorted(list(temp.values()), reverse = True)[:2])/sum(temp.values())
    else:
        return max(list(temp.values()))/sum(temp.values())
    
def radio_d_three(x):
    temp = Counter(x)
    if len(list(temp.values()))>=3:
        return sum(sorted(list(temp.values()), reverse = True)[:3])/sum(temp.values())
    elif len(list(temp.values()))>=2:
        return sum(sorted(list(temp.values()), reverse = True)[:2])/sum(temp.values())
    else:
        return max(list(temp.values()))/sum(temp.values())

# User aggregation characteristics
group_data = trans.groupby(['user'])['days_diff'].agg(
            user_centre_1d = radio_d,
            user_centre_2d = radio_d_two,
            user_centre_3d = radio_d_three
        ).reset_index()
data = data.merge(group_data, on=['user'], how='left')

''' 


TypeError: 'str' object does not support item assignment

In [50]:
temp = op.groupby(['user'])['days_diff'].agg(
            user_centre_1d = radio_d,
            user_centre_2d = radio_d_two,
            user_centre_3d = radio_d_three
        ).reset_index()
data = pd.merge(data, temp, on=['user'], how='left')

good_cols = list(data.columns)

for col in data.columns:
    rate = data[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.99:
        good_cols.remove(col)
        print(col, rate)

data = data[good_cols]

data.to_csv("Basic_feature.csv", index = False)


agreement1 0.9992905532293739
product4_amount 0.9918344067773033
product5_amount 0.9974126058953635
user_trans_days_nuniq 1.0
city_cnt 1.0
user_platform_80f6d63f26a56315_min_day 0.9998469820690806
user_platform_80f6d63f26a56315_max_day 0.9998469820690806
user_platform_80f6d63f26a56315_Is_recent_trd_mean 0.999833071348088
user_platform_fe8686492bb72dd4_min_day 0.9984141778068357
user_platform_fe8686492bb72dd4_max_day 0.9984141778068357
user_platform_fe8686492bb72dd4_Is_recent_trd_mean 0.9983446242018724
user_platform_fe8686492bb72dd4_gap_day 0.9984141778068357
user_type1_71b24e4fd9a658ee_min_day 0.9982611598759163
user_type1_71b24e4fd9a658ee_max_day 0.9982611598759163
user_type1_71b24e4fd9a658ee_Is_recent_trd_mean 0.9981776955499604
user_type1_71b24e4fd9a658ee_gap_day 0.9982611598759163
user_type1_c68ff01f3f65797d_min_day 0.9999721785580147
user_type1_b26bc49195bd79cf_min_day 0.9986784815056965
user_type1_b26bc49195bd79cf_max_day 0.9986784815056965
user_type1_b26bc49195bd79cf_Is_recent_

## Op related features

In [68]:
def cal_time(x):
    day = int(x.split(' ')[0])
    hour = int(x.split(' ')[2].split('.')[0].split(':')[0])
    minute = int(x.split(' ')[2].split('.')[0].split(':')[1])
    second = int(x.split(' ')[2].split('.')[0].split(':')[2])
    return 86400*day+3600*hour+60*minute+second

def generate_time(op):
    op['days_diff'] = op['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
    op['timestamp'] = op['tm_diff'].apply(lambda x: cal_time(x))
    op['hour'] = op['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
    return op

op=generate_time(op)
trans=generate_time(trans)


op['index']=op.index   
# user operation times
table = pd.pivot_table(op, values=['index'], index=['user'], aggfunc={pd.Series.nunique})  
table.columns = [' '.join(col).strip() for col in table.columns.values]  
table=table.reset_index()   
data=data[['user','label']].copy()  
table=table.merge(data[['user','label']],how='left',on=['user'])  
op=op.drop(columns=['index'])  
table=table.rename(columns={'index nunique':'op_times'})  
data=data.merge(table[['user','op_times']],how='left',on=['user'])  
data['op_times']=data['op_times'].fillna(0)  
data.dropna()  
trans_data=pd.read_csv('train_trans.csv')
trans_test=pd.read_csv('testb_trans.csv')  
trans=pd.concat([trans_data,trans_test],axis=0)   
# user transaction times
trans['index']=trans.index
table2 = pd.pivot_table(trans, values=['index'], index=['user'], aggfunc={pd.Series.nunique})  
table2.columns = [' '.join(col).strip() for col in table2.columns.values]
table2=table2.reset_index()  
table2=table2.rename(columns={'index nunique':'trans_times'})  
data=data.merge(table2[['user','trans_times']],how='left',on='user')  
data.dropna()  
data['trans_times']=data['trans_times'].fillna(0)  
data['op_per_trans']=data['op_times']/data['trans_times']  
data['op_per_trans']=data['op_per_trans'].fillna(0)  
data.dropna()   
op=generate_time(op)  
op=op.sort_values(by=['user','timestamp']) 

 #################### days which have operation records##########
table = pd.pivot_table(op, values=['days_diff'], index=['user'], aggfunc={pd.Series.nunique})  
table.columns = [' '.join(col).strip() for col in table.columns.values]  
table=table.reset_index()  
table=table.merge(data[['user','label']],how='left',on=['user'])  
data=data.merge(table[['user','days_diff nunique']],how='left',on=['user'])  
data['days_diff nunique']=data['days_diff nunique'].fillna(0)  

########### average operation times #########
data.dropna()  
data['op_per_day']=data['trans_times']/data['days_diff nunique']  
data['op_per_day']=data['op_per_day'].fillna(0)  

########### user operation days related features ########
op['days_diff'].astype(int)
op_days_unique=op.drop_duplicates(subset=['user','days_diff'])  
tmp = op_days_unique.groupby('user', as_index=False)['days_diff'].agg({list})  
tmp=tmp.reset_index()  
tmp=tmp.merge(data[['user','label']],how='left',on=['user'])   

def mean_interval(nums):
    sum_num=0
    for i in range(len(nums)-1):
        sum_num+=nums[i+1]-nums[i]
    if len(nums)==1:
        return 0
    return sum_num/(len(nums)-1) 
tmp['mean_interval']=tmp['list'].apply(lambda x:mean_interval(x))  

tmp['deviation']=tmp['list'].apply(lambda x:np.std(x))   

tmp['mean_value']=tmp['list'].apply(lambda x:np.mean(x))   

def max_interval(nums):
    max_interval=1
    for i in range(len(nums)-1):
        max_interval=max(max_interval,nums[i+1]-nums[i])
    return max_interval 
tmp['max_interval']=tmp['list'].apply(lambda x:max_interval(x))  

############# The time difference between the date of the last operation and the last day
tmp['last_op_day']=tmp['list'].apply(lambda x:np.max(x))  
data=data.merge(tmp[['user','mean_interval', 'deviation', 'mean_value',
       'max_interval']],how='left',on=['user'])  
data=data.merge(tmp[['user','last_op_day']],how='left',on=['user'])  
data['mean_interval']=data['mean_interval'].fillna(15)
data['max_interval']=data['max_interval'].fillna(15)
data['deviation']=data['deviation'].fillna(0)
data['last_op_day']=data['last_op_day'].fillna(0)  
data['mean_value']=data['mean_value'].fillna(0)
data.dropna()   

################the number of missing records for ip accounts for the total number of records
op['ip_3']=op['ip_3'].fillna(-1)  
op_na=op[op['ip_3']==-1]  
op_na['index']=op_na.index
table = pd.pivot_table(op_na, values=['index'], index=['user'], aggfunc={pd.Series.nunique})  
table.columns = [' '.join(col).strip() for col in table.columns.values]  
table=table.reset_index()  
table=table.merge(data[['user','label']],how='right',on=['user'])  
table['index nunique']=table['index nunique'].fillna(0)  
table=table.rename(columns={'index nunique':'na_ip_times'})  
data=data.merge(table[['user','na_ip_times']],how='left',on=['user'])  
data['na_ip_times']=data['na_ip_times'].fillna(0)  
data['nan_ip_percent']=data['na_ip_times']/data['na_ip_times']  
data['nan_ip_percent']=data['nan_ip_percent'].fillna(0)   
# Strong feature gain
data['mean_interval*deviation']=data['mean_interval']*data['deviation']  
data['mean_interval*nan_ip_per']=data['mean_interval']*data['nan_ip_percent'] 

# proportion of each operation type to the total number of operations
op['index']=op.index
table = pd.pivot_table(op[op['op_type']=='af1d68e0408f5148'], values=['index'], index=['user'], aggfunc={pd.Series.nunique})  
table.columns = [' '.join(col).strip() for col in table.columns.values]  
table=table.reset_index()  
table=table.rename(columns={'index nunique':'op_type1_times'})
  
data=data.merge(table,how='left',on='user')  
data['op_type1_times']=data['op_type1_times'].fillna(0)  
data['op_type1_times_percent']=data['op_type1_times']/data['op_times']  
data['op_type1_times_percent']=data['op_type1_times_percent'].fillna(0)  
op['index']=op.index
table = pd.pivot_table(op[op['op_type']=='b26bc49195bd79cf'], values=['index'], index=['user'], aggfunc={pd.Series.nunique})  
table.columns = [' '.join(col).strip() for col in table.columns.values]  
table=table.reset_index()  
table=table.rename(columns={'index nunique':'op_type2_times'})  
data=data.merge(table,how='left',on='user')  
data['op_type2_times']=data['op_type2_times'].fillna(0)  
data['op_type2_times_percent']=data['op_type2_times']/data['op_times']  
data['op_type2_times_percent']=data['op_type2_times_percent'].fillna(0)  
data.dropna()  

data['is_inf']=data.apply(lambda row:math.isinf(row['op_per_day']),axis=1)  
data.loc[data['is_inf']==True,'op_per_day']=0  
data=data.drop(columns=['is_inf'])  

data=data.drop(columns=['days_diff','timestamp','hour'])

data['sum_card_number']=data['card_a_cnt']+data['card_b_cnt']+data['card_c_cnt']+data['card_d_cnt']

data['card_a_percent']=data['card_a_cnt']/data['sum_card_number']
data['card_b_percent']=data['card_b_cnt']/data['sum_card_number']
data['card_c_percent']=data['card_c_cnt']/data['sum_card_number']
data['card_d_percent']=data['card_d_cnt']/data['sum_card_number']

data['card_a_percent']=data['card_a_percent'].fillna(0)
data['card_b_percent']=data['card_b_percent'].fillna(0)
data['card_c_percent']=data['card_c_percent'].fillna(0)
data['card_d_percent']=data['card_d_percent'].fillna(0)

data['op_count']=data['op1_cnt']+data['op2_cnt']
data['op1_percent']=data['op1_cnt']/data['op_count']
data['op2_percent']=data['op2_cnt']/data['op_count']
data['op1_percent']=data['op1_percent'].fillna(0)
data['op2_percent']=data['op2_percent'].fillna(0)

data['service_cnt']=data['service1_cnt']+data['service2_cnt']
data['service1_percent']=data['service1_cnt']/data['service_cnt']
data['service2_percent']=data['service2_cnt']/data['service_cnt']
data['service1_percent']=data['service1_percent'].fillna(0)
data['service2_percent']=data['service2_percent'].fillna(0)

# Percentage of different types of balance grades
data['balance1_avg_per']=data['balance1_avg']/data['balance_avg']
data['balance2_avg_per']=data['balance2_avg']/data['balance_avg']

# percentage of different products with different amount levels
data['product_amount']=data['product1_amount']+data['product2_amount']+data['product3_amount']+data['product4_amount']+data['product5_amount']+data['product6_amount']
for i in range(1,7):
    column_name='product'+str(i)+'_amount'
    data[column_name+'_percent']=data[column_name]/data['product_amount']
    data[column_name+'_percent']=data[column_name+'_percent'].fillna(0)


data['login_cnt_period_std']=data.apply(lambda row:np.std([row['login_cnt_period1'],row['login_cnt_period2']]),axis=1)

data['login_period1_percent']=data['login_cnt_period1']/(data['login_cnt_period1']+data['login_cnt_period2'])
data['login_period2_percent']=data['login_cnt_period2']/(data['login_cnt_period1']+data['login_cnt_period2'])
data['login_period1_percent']=data['login_period1_percent'].fillna(0)
data['login_period2_percent']=data['login_period2_percent'].fillna(0)

In [57]:
df = pd.read_csv('Basic_feature.csv')

col1 = list(df.columns)
col2 = list(data.columns)
column = [x for x in col2 if x not in col1]
data = data[column+['user']]

data = pd.merge(data, df, on = 'user', how = 'right')

In [64]:
good_cols = list(data.columns)

for col in data.columns:
    rate = data[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.99:
        good_cols.remove(col)
        print(col, rate)

data = data[good_cols]

data.head()

user_platform_80f6d63f26a56315_amount_count 0.999833071348088
user_platform_fe8686492bb72dd4_amount_count 0.9983446242018724
user_platform_80f6d63f26a56315_amount_sum 0.999833071348088
user_platform_fe8686492bb72dd4_amount_sum 0.9983446242018724
user_type1_71b24e4fd9a658ee_amount_count 0.9981776955499604
user_type1_81abaafd1ae512dd_amount_count 0.9968979092186347
user_type1_b26bc49195bd79cf_amount_count 0.9986367493427184
user_type1_c68ff01f3f65797d_amount_count 0.9999721785580147
user_type1_cd31880f9fa923ea_amount_count 0.999749607022132
user_type1_fc9b75cf62ba8b8f_amount_count 0.9995687676492273
user_type1_71b24e4fd9a658ee_amount_sum 0.9981776955499604
user_type1_81abaafd1ae512dd_amount_sum 0.9968979092186347
user_type1_b26bc49195bd79cf_amount_sum 0.9986367493427184
user_type1_c68ff01f3f65797d_amount_sum 0.9999721785580147
user_type1_cd31880f9fa923ea_amount_sum 0.999749607022132
user_type1_fc9b75cf62ba8b8f_amount_sum 0.9995687676492273
user_type2_08d5d580f39e34e6_amount_count 0.99994

,trans_times,op_per_trans,days_diff nunique,op_per_day,mean_interval,deviation,mean_value,max_interval,last_op_day,na_ip_times,...,op_times,user_b2e7fa260df4998d_cnt,user_b2e7fa260df4998d_ratio,user_116a2503b987ea81_cnt,user_116a2503b987ea81_ratio,user_b131ac74aa38a121_cnt,user_b131ac74aa38a121_ratio,user_centre_1d,user_centre_2d,user_centre_3d
0,4.0,10.500000,3.0,1.333333,1.50,1.247219,12.333333,2.0,14.0,7.0,...,12.0,NaN,NaN,12.0,1.000000,NaN,NaN,0.904762,0.928571,0.952381
1,2.0,1.500000,1.0,2.000000,0.00,0.000000,11.000000,1.0,11.0,0.0,...,3.0,NaN,NaN,3.0,1.000000,NaN,NaN,0.666667,1.000000,1.000000
2,9.0,1.555556,1.0,9.000000,0.00,0.000000,6.000000,1.0,6.0,4.0,...,5.0,1.0,0.200000,NaN,NaN,4.0,0.8,0.928571,1.000000,1.000000
3,18.0,3.222222,9.0,2.000000,1.25,3.424674,4.777778,2.0,10.0,19.0,...,44.0,1.0,0.022727,21.0,0.477273,22.0,0.5,0.775862,0.810345,0.844828
4,1.0,11.000000,2.0,0.500000,3.00,1.500000,11.500000,3.0,13.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.818182,0.909091,1.000000


Word2Vec for amount, op_type...(text) feature

In [74]:
def emb(df, f1, f2):
    # size represents the dimensionality of the output feature vector, which is generally taken to be between a few tens and a few hundred
    emb_size = 64
    print('====================================== {} {} ======================================'.format(f1, f2))

    df[f2]=df[f2].fillna('0')
    
    # Data sorting, first keyword user, second keyword timestamp
    df=df.sort_values(by=['user','timestamp'])
    tmp = df.groupby(f1, as_index=False)[f2].agg({'{}_{}_list'.format(f1, f2): list})
    sentences = tmp['{}_{}_list'.format(f1, f2)].values.tolist()
    del tmp['{}_{}_list'.format(f1, f2)]
    for i in range(len(sentences)):
        sentences[i] = [str(x) for x in sentences[i]]
    model = Word2Vec(sentences, vector_size=emb_size, window=5, min_count=3, sg=0, hs=1, seed=2222)
    emb_matrix = []
    for seq in sentences:
        vec = []
        for w in seq:
            if w in model.wv:
                vec.append(model.wv[w])
        if len(vec) > 0:
            emb_matrix.append(np.mean(vec, axis=0))
        else:
            emb_matrix.append([0] * emb_size)
    emb_matrix = np.array(emb_matrix)
    for i in range(emb_size):
        tmp['{}_{}_emb_{}'.format(f1, f2, i)] = emb_matrix[:, i]
    del model, emb_matrix, sentences
    return tmp

# Transaction Amount

emb_cols = [
    ['user', 'amount']
]

amount_w2v=pd.DataFrame(data['user'].copy())

trans=generate_time(trans)

for f1, f2 in emb_cols:
    amount_w2v = amount_w2v.merge(emb(trans, f1, f2), on=f1, how='left')

#%%
amount_w2v=amount_w2v.drop(columns=['user'])
amount_w2v['sum_op_w2v']=amount_w2v.sum(axis=1)

data=pd.concat([data,amount_w2v['sum_op_w2v']],axis=1)

#%% operation data
emb_cols = [
    ['user','op_type'],
    ['user','op_mode'],
    ['user','op_device'],
    ['user','ip'],
    ['user','net_type'],
    ['user','channel'],
    ['user','ip_3'],
]

op=generate_time(op)

op_w2v=pd.DataFrame(data['user'].copy())

for f1, f2 in emb_cols:
    op_w2v=emb(op, f1, f2)
    op_w2v=op_w2v.drop(columns=['user'])
    column_name='sum_w2v'+f2
    op_w2v[column_name]=op_w2v.sum(axis=1)
    data=pd.concat([data,op_w2v[column_name]],axis=1)

====================================== user amount ======================================
====================================== user op_type ======================================
====================================== user op_mode ======================================
====================================== user op_device ======================================
====================================== user ip ======================================
====================================== user net_type ======================================
====================================== user channel ======================================
====================================== user ip_3 ======================================


In [77]:
data.head()

,trans_times,op_per_trans,days_diff nunique,op_per_day,mean_interval,deviation,mean_value,max_interval,last_op_day,na_ip_times,...,user_centre_2d,user_centre_3d,sum_op_w2v,sum_w2vop_type,sum_w2vop_mode,sum_w2vop_device,sum_w2vip,sum_w2vnet_type,sum_w2vchannel,sum_w2vip_3
0,4.0,10.500000,3.0,1.333333,1.50,1.247219,12.333333,2.0,14.0,7.0,...,0.928571,0.952381,-4.637256,-8.136339,-0.475300,2.985004,-5.030328,4.763794,7.440312,-2.014670
1,2.0,1.500000,1.0,2.000000,0.00,0.000000,11.000000,1.0,11.0,0.0,...,1.000000,1.000000,0.111074,-5.972022,-0.836419,1.776785,0.502700,5.272960,6.264074,5.748219
2,9.0,1.555556,1.0,9.000000,0.00,0.000000,6.000000,1.0,6.0,4.0,...,1.000000,1.000000,-2.832696,-1.755416,7.076905,9.253852,3.914774,5.354793,6.183161,2.786482
3,18.0,3.222222,9.0,2.000000,1.25,3.424674,4.777778,2.0,10.0,19.0,...,0.810345,0.844828,0.516070,-6.728838,-0.591093,4.799207,4.165147,3.778757,1.146504,-10.967967
4,1.0,11.000000,2.0,0.500000,3.00,1.500000,11.500000,3.0,13.0,1.0,...,0.909091,1.000000,0.320182,-1.100410,-0.557338,1.752873,-1.629506,-0.095596,-0.823160,8.272836


In [78]:
#data = fill_discrete_conti_na(data)
data.to_csv("data_feature_final.csv")

# Model


In [81]:
def lgb_model(train, target,test, k):
    output_preds = 0
    feats = [f for f in train.columns if f not in ['user', 'label']]
    print('Current num of features:', len(feats))
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)
    oof_probs = np.zeros(train.shape[0])
    offline_score = []
    feature_importance_df = pd.DataFrame()
    parameters = {
        'learning_rate': 0.01,
        'reg_lambda':1.9,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 64,
        'max_depth':8,
        'feature_fraction': 0.3,
        'bagging_fraction': 0.7,
        'min_data_in_leaf': 20,
        'verbose': -1,
        'nthread': 4
    }

    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]

        dtrain = lgb.Dataset(train_X,
                             label=train_y)
        dval = lgb.Dataset(test_X,
                           label=test_y)
        lgb_model = lgb.train(
                parameters,
                dtrain,
                num_boost_round=5000,
                valid_sets=[dval],
                early_stopping_rounds=150,
                verbose_eval=100,
        )
        
        oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration)
        offline_score.append(lgb_model.best_score['valid_0']['auc'])
        print(offline_score)
        
        output_preds += lgb_model.predict(test[feats], num_iteration=lgb_model.best_iteration)/folds.n_splits

        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('OOF-MEAN-AUC:%.6f',(np.mean(offline_score)))
    return np.mean(offline_score),feature_importance_df,output_preds

train=data[:47782]
test=data.iloc[47782:,:]
test=test.reset_index()
test=test.drop(columns=['index'])

mean_auc,feature_imp,pred=lgb_model(train,train['label'],test,5)

table = pd.pivot_table(feature_imp, values=['importance'], index=['feature'], aggfunc={np.mean})
table.columns = [' '.join(col).strip() for col in table.columns.values]

table=table.sort_values(by=['importance mean'],ascending=False)
table=table.reset_index()

table.to_csv('feature_imp.csv',index=False)

important_feature=np.array(table.loc[:350,'feature']).tolist()
train_after_feature_selection=train[important_feature]
test_after_feature_selection=test[important_feature]
all_after_feature_selection=pd.concat([train_after_feature_selection,test_after_feature_selection],axis=0)
print(len(train_after_feature_selection.columns))
mean_auc,feature_imp,pred=lgb_model(train_after_feature_selection,train['label'],all_after_feature_selection,5)

import datetime
result=pd.concat([test['user'],pd.DataFrame(pred[47782:],columns=['prob'])],axis=1)
result.to_csv('pred_'+str(len(important_feature))+'.csv',index=False)

Current num of features: 486
Training until validation scores don't improve for 150 rounds
[100]	valid_0's auc: 0.698224
[200]	valid_0's auc: 0.704796
[300]	valid_0's auc: 0.708892
[400]	valid_0's auc: 0.711597
[500]	valid_0's auc: 0.713495
[600]	valid_0's auc: 0.714659
[700]	valid_0's auc: 0.715557
[800]	valid_0's auc: 0.716263
[900]	valid_0's auc: 0.716706
[1000]	valid_0's auc: 0.716935
[1100]	valid_0's auc: 0.717368
[1200]	valid_0's auc: 0.717466
[1300]	valid_0's auc: 0.717659
[1400]	valid_0's auc: 0.717611
Early stopping, best iteration is:
[1299]	valid_0's auc: 0.71767
[0.7176699661360936]
Training until validation scores don't improve for 150 rounds
[100]	valid_0's auc: 0.713842
[200]	valid_0's auc: 0.72262
[300]	valid_0's auc: 0.72691
[400]	valid_0's auc: 0.729911
[500]	valid_0's auc: 0.731822
[600]	valid_0's auc: 0.733348
[700]	valid_0's auc: 0.734141
[800]	valid_0's auc: 0.73454
[900]	valid_0's auc: 0.734698
[1000]	valid_0's auc: 0.734587
Early stopping, best iteration is:
[92